# A* Chat

As your application grows bigger, you may want to create more than one agent, each with a different set of tools to handle different part of the problem. Orchestrating a conversation between these agents can be challenging, especially when it is difficult to determind which agent should be called at each timestep.

Seeing multi-agent chat as a path finding problem, where each message in a message history is analogous to a node, and the message history as the path, A* algorithm can be used to find the optimal path to the end of the conversation.

A* algorithm is a path finding algorithm that is widely used. It is a variant of Dijkstra's algorithm, which is used to find the shortest path between two nodes in a graph. A* algorithm is an extension of Dijkstra's algorithm, which adds a heuristic function to guide the search towards the goal. The heuristic function is an estimation of the distance between the current node and the goal. The algorithm will always choose the node with the lowest cost, which is the sum of the distance from the start node to the current node and the heuristic function.

Using A* Chat, instead of having to manually program agent behaviours, you can simply define the heuristic function that estimates how *close* the message history to the goal, and the algorithm will automatically orchestrate the conversation between the agents to reach the goal.

## Example 0: A* Chat to write an email that resonate the audience based on their MBTI personality type
We will use A* Chat to orchestrate a conversation between a three agents:
- writer: writes the product pitch
- editor: write edits the product pitch according to the critics' feedback
- critics: gives feedback to the story

In [ ]:
from agentx.agent import Agent
from agentx.schema import GenerationConfig, Function, Message, Content
from agentx.astar import astarchat
from pydantic import BaseModel, Field
from typing import List

# define the agents and the tools they can use

writer_agent = Agent(
    name='geocoding_agent',
    system_prompt='Use the functions you have been provided to solve the problem. Reply TERMINATE to end the conversation when the problem is solved.',
    generation_config=GenerationConfig(
        api_type='azure',
        api_key=os.environ.get('AZURE_OPENAI_KEY'),
        base_url=os.environ.get('AZURE_OPENAI_ENDPOINT'),
        azure_deployment='gpt-35',
        tools={
            geocoding.name:Function(
                name=geocoding.name,
                description=geocoding.description,
                parameters=geocoding.input_json_schema,
            )
        }
    ),
    function_map={
        geocoding.name: geocoding.run
    },
    reduce_function=lambda x: x[-1],
)

geodesic_calculation_agent = Agent(
    name = 'geodesic_calculation_agent',
    system_prompt='Use the functions you have been provided to solve the problem. Reply TERMINATE to end the conversation when the problem is solved.',
    generation_config=GenerationConfig(
        api_type='azure',
        api_key=os.environ.get('AZURE_OPENAI_KEY'),
        base_url=os.environ.get('AZURE_OPENAI_ENDPOINT'),
        azure_deployment='gpt-35',
        tools={
            geodesic.name:Function(
                name=geodesic.name,
                description=geodesic.description,
                parameters=geodesic.input_json_schema,
            )
        }
    ),
    function_map={
        geodesic.name: geodesic.run
    },
    reduce_function=lambda x: x[-1],
)

# At each timestep, A* minimize heuristic + cost
# heuristic: an estimation of the distance between the current state and the goal state
# cost: the distance between the start state and the current state

# In this example, heuristic is whether the geodesic distance is calculated correctly

class HeuristicScore(BaseModel):
    score:float = Field(0, ge=0, le=10)

heuristic_agent = Agent(
    name='heuristic',
    generation_config=GenerationConfig(
        api_type='azure',
        api_key=os.environ.get('AZURE_OPENAI_KEY'),
        base_url=os.environ.get('AZURE_OPENAI_ENDPOINT'),
        azure_deployment='gpt-35',
    ),
)

def heuristic(messages:List[Message]) -> float:
    score = heuristic_agent.generate_response(
        [
            Message(
                role='user',
                content=Content(
                    text='Base on this chat history: {history}'.format(
                        history=[message.model_dump_json(
                            exclude_unset=True, 
                            exclude_none=True
                        ) for message in messages]
                    ),
                ),
            )
        ] + [
            Message(
                role='user',
                content=Content(
                    text='Estimate the progress of solving the problem. Give a score of 10 to represent that the geodesic distance is calculated correctly by acquiring all the available information. You must reply an JSON object.',
                ),
            )
        ],
        output_model=HeuristicScore # the output model is used to validate the response
    )
    return 10 - HeuristicScore.model_validate_json(score.content.text).score

# Cost is the number of LLM calls
def cost(messages:List[Message], next_message:List[Message]) -> float:
    # tool calls are not counted
    cost = sum([
        1 for message in next_message if message.role != 'tool'
    ])
    return cost

In [ ]:

reconstructed_path, came_from, cost_so_far, heuristic_map, hash_map = await astarchat(
    agents = [geocoding_agent, geodesic_calculation_agent],
    heuristic = heuristic,
    cost = cost,
    messages = [
        Message(
            role='user',
            content=Content(
                text='What is the distance between Gare Port La Goulette - Sud in Tunisia and Porto di Napoli in Italy?',
            ),
        )
    ],
    threshold = 1,
    n_replies = 1,
    max_iteration = 10,
    max_queue_size = 10
)

print(reconstructed_path)
print(came_from)
print(cost_so_far)
print(heuristic_map)
print(hash_map)